In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras import optimizers

# generator

keras에 넣을려고 만든 generator입니다
1. data: numpy array꼴
2. lookback: timestep
3. min_index, max_index: train data 안에서 validation data를 따로 만들어서 써보고 싶을때 사용. 통째로 train하려면 max_index = None
4. batch_size: batch_size
5. step=1, delay=0: autoencoder에서는 그냥 기본값으로.. 

In [ ]:
def generator(data, lookback, min_index, max_index, batch_size,
             shuffle=False, step=1, delay=0):
    
    if max_index is None:
        max_index = len(data) - delay -1
        
    i = min_index + lookback
    
    while 1:
        if shuffle:
            rows = np.random.randint(min_index+lookback, max_index, size=batch_size)
    # min_index+lookback과 max_index 사이의 숫자를 batch_size만큼 만든다.
    # 단순히 batch를 만들어주기 위해 index를 rows에 저장  
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            
            rows = np.arrange(i, min(i+batch_size, max_index))
            i += len(rows)
        
        samples = np.zeros((len(rows), lookback//step , data.shape[-1]))
        targets = np.zeros((len(rows), lookback//step , data.shape[-1]))
        # shape = (batchsize, lookback, feature 수)
        
        for j, row in enumerate(rows):# j 는 0~batchsize, row는 index number
            indices = range(rows[j]-lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[indices]
        
        yield samples, targets

밑에 train_np는 normalization 다 시키고, numpy 배열화 한 데이터

In [ ]:
train_gen = generator(train_np, lookback = 90, min_index = 0, max_index=700000, batch_size= 200, shuffle=True)
valid_gen = generator(train_np, lookback = 90, min_index = 700001, max_index=None, batch_size= 200, shuffle=True)

In [ ]:
val_steps = ((len(train_np) -1) - 700001 - 90) // 200

# autoencoder + lstm

모델 학습

In [ ]:
ts = 90

model = Sequential()

model.add(layers.LSTM(128, activation='tanh',
                input_shape=(ts,79), return_sequences=True))
model.add(layers.LSTM(64, activation='tanh', return_sequences=False))
## encoder
model.add(layers.RepeatVector(ts))
model.add(layers.LSTM(64, activation='tanh', return_sequences=True))
model.add(layers.LSTM(128, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(79)))
# activation = relu로 하면 lstm이 gpu로 안돌아가네요..
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit_generator(train_gen, steps_per_epoch = 500, 
                     epochs=20, validation_data = valid_gen, validation_steps=val_steps)

# Validation data

validation data는 data가 별로 안커서 직접 저장이 되더라구요

In [ ]:
def create_sequences(values, time_steps=ts):
    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i : (i + time_steps)])
    # Convert 2D sequences into 3D as we will be feeding this into
    # a convolutional layer.
    return np.expand_dims(output, axis=2)

In [ ]:
valid1 = create_sequences(VALID_NP)

In [ ]:
valid1 = valid1.reshape((valid1.shape[0],valid1.shape[1],valid1.shape[3]))

In [ ]:
preds = model.predict(valid1)